<a href="https://colab.research.google.com/github/jaswanthreddy049/GEN_AI/blob/main/INITIAL_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama_index gradio chromadb sentence-transformers
!pip install llama-index-llms-groq
!pip install llama-index-vector-stores-chroma
!pip install -U llama-index
!pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.groq import Groq
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

In [6]:
from google.colab import files
uploaded = files.upload()

Saving hr.pdf to hr (1).pdf


In [7]:

filename = list(uploaded.keys())[0]
reader = PDFReader()
documents = reader.load_data(filename)
#use unstructured   ``

In [8]:

parser = SentenceSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
nodes = parser.get_nodes_from_documents(documents)

In [11]:

# Load the BGE-Large v1.5 model
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

# Generate embeddings for the split documents
# Assuming split_documents is a list of LlamaIndex TextNode objects
embeddings_list = embedding_model.get_text_embedding_batch([node.text for node in nodes])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [12]:


chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection("my_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex(
    nodes,
    vector_store=vector_store,
    embed_model=embedding_model  # e.g. BGE-large
)

In [13]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5  # number of chunks to retrieve
)

In [18]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('demo')


llm = Groq(
    model="llama3-70b-8192",  # or 'mixtral-8x7b-32768'
    temperature=0.3,
)


In [20]:

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",  # or "simple", "tree_summarize", etc.
    llm=llm
)


In [22]:


query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)


In [24]:
query = "what are the benifits for new employee."

response = query_engine.query(query)
print(response)


As a new employee, you have a 60-day wait for health insurance through UMMS. During this waiting period, you can research options through the Massachusetts Health Connector at www.mahealthconnector.org. You can enroll and pay online by the 10th of the previous month to be covered for the first of a month. Additionally, you will need to notify the Connector when you wish your coverage to terminate.


In [25]:
import gradio as gr

def ask_bot(query):
    return query_engine.query(query)

gr.Interface(fn=ask_bot, inputs="text", outputs="text", title="EduQuery").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98ff0963982deea69d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
